<a href="https://colab.research.google.com/github/imai-ry/colabo/blob/main/AIS_RNN_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIS進路予測

# データ読み込み

In [1]:
# ライブラリのインストール
# folium : 地図の表示
!python3 -m pip install folium

In [2]:
# AISデータの読み込み

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/colab-data/logcsv/aislog.csv', index_col=0)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,mmsi,sog,x,y,cog,true_heading,time
0,3,431013747,11.2,139.729398,35.155807,209.300003,208.0,2021-03-29 06:17:02.075856
1,3,431101119,8.6,139.783410,35.179812,182.699997,187.0,2021-03-29 06:17:02.134460
2,3,431004957,0.1,139.736633,35.393172,153.100006,175.0,2021-03-29 06:17:02.237737
3,1,431000932,9.7,139.597435,35.069915,241.300003,241.0,2021-03-29 06:17:03.221118
4,3,431000124,13.0,139.751620,35.198780,3.500000,3.0,2021-03-29 06:17:03.434945
...,...,...,...,...,...,...,...,...
48273,1,431011759,12.3,139.746050,35.194928,213.600006,218.0,2021-03-29 09:17:00.688802
48274,3,431501813,10.3,139.732350,35.162033,210.000000,211.0,2021-03-29 09:17:00.717955
48275,1,636019422,13.9,139.725133,35.115733,214.000000,218.0,2021-03-29 09:17:01.039736
48276,3,431999603,12.0,139.741132,35.089850,296.500000,293.0,2021-03-29 09:17:01.168476


In [3]:
# 欠損値NaN除去
ais_df = df.dropna(how="any")

# MMSI別に抽出
targets = [ais_df[ais_df["mmsi"]==mmsi] for mmsi in ais_df["mmsi"].unique()]
print(len(targets))

222


In [4]:
# target0を例に図示
# xyのみ抽出
xy = targets[0].loc[:, ["x","y"]]

# 緯度経度を入れ替えて、pythonリストに変換
latlngs = xy.values[:,[1,0]].tolist()
latlngs[:5]

# 航跡の表示
import folium
m = folium.Map(location=[35.3, 140], zoom_start=10)
polyline = folium.PolyLine(latlngs, color="black")
polyline.add_to(m)
m

# データセットの作成

In [7]:
import numpy as np
timesteps = 10   # 学習に利用するAISの数
n_features = 4  # x,y,cog, sog

def create_dataset(targets, n_features, timesteps):
  x = np.zeros((0,timesteps,n_features))
  y = np.zeros((0,2))
  for i in range(len(targets)):
    lng = targets[i]["x"].tolist()
    lat = targets[i]["y"].tolist()
    cog = targets[i]["cog"].tolist()
    sog = targets[i]["sog"].tolist()
    state = np.array([lng, lat, sog, cog]).T
    n_trajectory_points, n_features = state.shape
    for i in range(n_trajectory_points-timesteps):
      tx = state[i:i+timesteps, :]
      ty = state[i+timesteps, :2]
      x = np.append(x, tx.reshape(1,tx.shape[0],tx.shape[1]), axis=0)
      y = np.append(y, ty.reshape(1,ty.shape[0]), axis=0)
  return x, y

## テストデータ


データを正規化する

In [8]:
# targets[0](一隻分の航跡)をテストデータに
test_targets = [targets[0]]
test_x, test_y = create_dataset(test_targets, n_features, timesteps)

test_x[:,:,:2] = (test_x[:,:,:2] - np.array([139.7, 35.1])) * 10
test_y[:,:] = (test_y[:,:] - np.array([139.7, 35.1])) * 10

print(f"test_x.shape:{test_x.shape}")
print(f"test_x:\n{test_x[:2,:,:]}")
print(f"test_y.shape:{test_y.shape}")
print(f"test_y:\n{test_y[:2,:]}")

test_x.shape:(871, 10, 4)
test_x:
[[[  0.29398333   0.55806667  11.19999981 209.30000305]
  [  0.29078333   0.55356667  11.39999962 211.19999695]
  [  0.28726667   0.54861667  11.5        210.3999939 ]
  [  0.28428333   0.54461667  11.60000038 211.8999939 ]
  [  0.28103333   0.54016667  11.39999962 210.1000061 ]
  [  0.27776667   0.5357      10.89999962 209.8999939 ]
  [  0.27453333   0.5312      11.10000038 212.        ]
  [  0.27125      0.52673333  11.60000038 211.69999695]
  [  0.26766667   0.5218      11.39999962 212.30000305]
  [  0.26471667   0.5178      11.         210.6000061 ]]

 [[  0.29078333   0.55356667  11.39999962 211.19999695]
  [  0.28726667   0.54861667  11.5        210.3999939 ]
  [  0.28428333   0.54461667  11.60000038 211.8999939 ]
  [  0.28103333   0.54016667  11.39999962 210.1000061 ]
  [  0.27776667   0.5357      10.89999962 209.8999939 ]
  [  0.27453333   0.5312      11.10000038 212.        ]
  [  0.27125      0.52673333  11.60000038 211.69999695]
  [  0.26766

## 訓練データ

In [ ]:
training_targets = targets[1:]
train_x, train_y = create_dataset(training_targets, n_features, timesteps)

train_x[:,:,:2] = (train_x[:,:,:2] - np.array([139.7, 35.1])) * 10
train_y[:,:] = (train_y[:,:] - np.array([139.7, 35.1])) * 10

print(f"船の総数：{len(targets)}")
print(f"training size: {len(training_targets)}")
print(f"test size: {len(test_targets)}")
print(f"timesteps: {timesteps}")
print(f"train_x.shape:{train_x.shape}, train_y.shape:{train_y.shape}")

print(f"train_x:\n{train_x[:2, :,:]}")
print(f"train_y:\n{train_y[:2, :]}")

船の総数：222
training size: 221
test size: 1
timesteps: 10
train_x.shape:(40143, 10, 4), train_y.shape:(40143, 2)
train_x:
[[[  0.8341       0.79811667   8.60000038 182.69999695]
  [  0.834        0.79688333   8.60000038 183.69999695]
  [  0.83385      0.79565      8.69999981 184.        ]
  [  0.83363333   0.79396667   8.89999962 184.69999695]
  [  0.83345      0.79275      8.89999962 185.8999939 ]
  [  0.83318333   0.7911       8.89999962 186.6000061 ]
  [  0.83295      0.78943333   9.         186.8999939 ]
  [  0.83218333   0.78521667   9.         187.6000061 ]
  [  0.8315       0.78151667   9.         188.30000305]
  [  0.83066667   0.77748333   8.89999962 189.30000305]]

 [[  0.834        0.79688333   8.60000038 183.69999695]
  [  0.83385      0.79565      8.69999981 184.        ]
  [  0.83363333   0.79396667   8.89999962 184.69999695]
  [  0.83345      0.79275      8.89999962 185.8999939 ]
  [  0.83318333   0.7911       8.89999962 186.6000061 ]
  [  0.83295      0.78943333   9.      

# モデルの定義と学習

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM

# モデル構築
units = 30
model = Sequential([
  LSTM(units, input_shape=(timesteps, n_features), return_sequences=False),
  Dense(2, activation="linear")      
])
model.compile(loss="mean_squared_error", optimizer="adam")
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 30)                4200      
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_x, train_y, batch_size=50, epochs=10, validation_split=0.2)

Epoch 1/10
283/283 [==============================] - 4s 8ms/step - loss: 0.3531 - val_loss: 0.2369
Epoch 2/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0804 - val_loss: 0.0677
Epoch 3/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0215 - val_loss: 0.0291
Epoch 4/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0106 - val_loss: 0.0182
Epoch 5/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0071 - val_loss: 0.0113
Epoch 6/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0054 - val_loss: 0.0083
Epoch 7/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0038 - val_loss: 0.0058
Epoch 8/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0027 - val_loss: 0.0034
Epoch 9/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0023 - val_loss: 0.0031
Epoch 10/10
283/283 [==============================] - 2s 6ms/step - loss: 0.0021 - val_loss: 0.0025

#テストデータに対する予測

In [ ]:
prediction = model.predict(test_x)

pred = prediction/10 + np.array([139.7, 35.1])
test_y = test_y/10 + np.array([139.7, 35.1])

pre_xy = pred[:,:].tolist()
test_xy = test_y[:,:].tolist()

print(f"真の航跡:\n{test_xy[:10]}")
print(f"予測航跡:\n{pre_xy[:10]}")

真の航跡:
[[139.72612, 35.15128833333333], [139.72582833333334, 35.150886666666665], [139.72550666666666, 35.150436666666664], [139.72519333333332, 35.14998333333333], [139.72483166666666, 35.14948666666667], [139.72452166666667, 35.14903666666667], [139.72443333333334, 35.14890166666667], [139.72443333333334, 35.14890166666667], [139.72422166666666, 35.148581666666665], [139.72392166666666, 35.14817166666667]]
予測航跡:
[[139.72718732878565, 35.14806812107563], [139.7264012571424, 35.147053024172784], [139.72611987665294, 35.14649624973536], [139.7258690472692, 35.14609651863575], [139.72576098963617, 35.14607439413667], [139.7251556932926, 35.14530554637313], [139.72464766427873, 35.14456463828683], [139.72454161085187, 35.14445857554674], [139.72438735961913, 35.144287238270046], [139.7241626445204, 35.14396775737405]]


In [ ]:
import folium
m = folium.Map(location=[35.3, 140], zoom_start=10)

pre_latlngs = [[y,x] for x,y in pre_xy]
pre_polyline = folium.PolyLine(pre_latlngs, color="red")
pre_polyline.add_to(m)

test_latlngs = [[y,x] for x,y in test_xy]
test_polyline = folium.PolyLine(test_latlngs, color="black")
test_polyline.add_to(m)

m